---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/blob/main/Applied_Text_Mining_in_Python/Week3/Labs/Assignment 3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>

In [ ]:
%%bash
# If running in Google Colab

# mkdir data
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Applied_Text_Mining_in_Python/Week3/Labs/data/spam.csv -P ./data

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('./data/spam.csv')
spam_data['target'] = np.where(spam_data['target'] == 'spam', 1, 0)
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], spam_data['target'], random_state=0)

### Question 1

What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    return spam_data['target'].mean() * 100 # Your answer here

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer().fit(X_train)
    tokens = [(token, len(token)) for token in vectorizer.vocabulary_.keys()]
    
    return sorted(tokens, key=lambda item: item[1], reverse=True)[0][0] #Your answer here

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

def answer_three():
    vectorizer = CountVectorizer().fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = model.predict(vectorizer.transform(X_test))
    
    return roc_auc_score(y_test, predictions) #Your answer here

answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series `(smallest tf-idfs series, largest tf-idfs series)`.*

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vectorizer = TfidfVectorizer().fit(X_train)
    feature_names_idfs = list(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

    smallest_idfs = sorted(feature_names_idfs, key=lambda item: item[1])[:20]
    largest_idfs = sorted(feature_names_idfs, key=lambda item: item[1], reverse=True)[:20]

    smallest_idfs = pd.Series([smallest_idf[1] for smallest_idf in smallest_idfs],
                              index=[smallest_idf[0] for smallest_idf in smallest_idfs])
    largest_idfs = pd.Series([largest_idf[1] for largest_idf in largest_idfs],
                              index=[largest_idf[0] for largest_idf in largest_idfs])
    
    return smallest_idfs, largest_idfs #Your answer here

smallest_tf_idfs_series, largest_tf_idfs_series = answer_four()

In [7]:
smallest_tf_idfs_series

to      2.198406
you     2.265645
the     2.707383
in      2.890761
and     2.976764
is      3.003012
me      3.111530
for     3.206840
it      3.222174
my      3.231044
call    3.297812
your    3.300196
of      3.319473
have    3.354130
that    3.408477
on      3.463136
now     3.465949
can     3.545053
are     3.560414
so      3.566625
dtype: float64

In [8]:
largest_tf_idfs_series

000pes         8.644919
0089           8.644919
0121           8.644919
01223585236    8.644919
0125698789     8.644919
02072069400    8.644919
02073162414    8.644919
02085076972    8.644919
021            8.644919
0430           8.644919
07008009200    8.644919
07099833605    8.644919
07123456789    8.644919
0721072        8.644919
07753741225    8.644919
077xxx         8.644919
078            8.644919
07808247860    8.644919
07808726822    8.644919
078498         8.644919
dtype: float64

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_five():
    vectorizer = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = model.predict(vectorizer.transform(X_test))
    
    return roc_auc_score(y_test, predictions) #Your answer here

answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [10]:
def answer_six():
    spam_data['length'] = spam_data['text'].apply(lambda txt: len(txt))
    not_spam_mean_length = np.mean(spam_data[spam_data['target'] == 0]['length'])
    spam_mean_length = np.mean(spam_data[spam_data['target'] == 1]['length'])
    
    return not_spam_mean_length, spam_mean_length # Your answer here

answer_six()

(71.02362694300518, 138.8661311914324)

The following function has been provided to help you combine new features into the training data:

In [11]:
from scipy.sparse import csr_matrix, hstack

def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

def answer_seven():
    vectorizer = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())
    
    model = SVC(C=10000).fit(X_train_vectorized, y_train)
    X_test_vectorized = add_feature(vectorizer.transform(X_test), X_test.str.len())
    predictions = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, predictions) # Your answer here

answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [13]:
def answer_eight():
    spam_data['digits'] = spam_data['text'].apply(lambda txt: len([ch for ch in txt if ch.isdigit()]))
    not_spam_mean_digits = np.mean(spam_data[spam_data['target'] == 0]['digits'])
    spam_mean_digits = np.mean(spam_data[spam_data['target'] == 1]['digits'])
    
    return not_spam_mean_digits, spam_mean_digits # Your answer here

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

def answer_nine():
    vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized, [X_train.str.len(), X_train.str.findall('\d').str.len()])
    X_test_vectorized = vectorizer.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, [X_test.str.len(), X_test.str.findall('\d').str.len()])

    model = LogisticRegression(C=100, max_iter=1000).fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, predictions) #Your answer here

answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [15]:
def answer_ten():
    spam_data['non_word'] = spam_data['text'].str.findall('\W').str.len()
    non_spam_mean_non_word = np.mean(spam_data[spam_data['target'] == 0]['non_word'])
    spam_mean_non_word = np.mean(spam_data[spam_data['target'] == 1]['non_word'])
    
    return non_spam_mean_non_word, spam_mean_non_word # Your answer here

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

Fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def answer_eleven():
    vectorizer = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized, [X_train.str.len(), X_train.str.findall('\d').str.len(), X_train.str.findall('\W').str.len()])
    X_test_vectorized = vectorizer.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, [X_test.str.len(), X_test.str.findall('\d').str.len(), X_test.str.findall('\W').str.len()])

    model = LogisticRegression(C=100, max_iter=1000).fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)
    auc_score = roc_auc_score(y_test, predictions)

    feature_names = np.array(vectorizer.get_feature_names_out().tolist() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    smallest = list(feature_names[sorted_coef_index[:10]])
    largest = list(feature_names[sorted_coef_index[:-11:-1]])
    
    return auc_score, smallest, largest # Your answer here

answer_eleven()

(0.9780231906694056,
 [' i', 'ca', '..', '. ', 'pe', ' go', ' m', 'if', 'us', 'go'],
 ['digit_count', 'ia', ' r', 'xt', 'ne', 'co', ' ba', ' x', '46', 'ian '])